In [1]:
import os
import pytorch_lightning as pl
import pandas as pd

from rdkit import Chem # safe import before ccdc imports
from torch_geometric.loader import DataLoader
from torch.utils.data import ConcatDataset

from conf_ensemble_dataset_in_memory import ConfEnsembleDataset
from data_split import DataSplit, MoleculeSplit, ProteinSplit
from bioschnet import BioSchNet
from molsize_model import MolSizeModel
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from tqdm import tqdm

In [2]:
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

In [3]:
pl.seed_everything(42, workers=True)

Global seed set to 42


42

# Data preparation

In [4]:
splits = ['random', 'scaffold', 'protein']

In [5]:
def get_loaders(data_split: DataSplit,
                batch_size: int=64) :
    
    train_dataset = ConfEnsembleDataset(data_split, 'train')
    val_dataset = ConfEnsembleDataset(data_split, 'val')
    test_dataset = ConfEnsembleDataset(data_split, 'test')

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
    
    return train_loader, val_loader, test_loader

In [ ]:
for split_type in splits :
    
    for split_i in range(5) :
    
        if split_type in ['random', 'scaffold'] :
            data_split = MoleculeSplit(split_type, split_i)
        elif split_type == 'protein' :
            data_split = ProteinSplit(split_type, split_i)
            
        batch_size = 128
        train_loader, val_loader, test_loader = get_loaders(data_split, batch_size)
        
        experiment_name = f'{split_type}_split_{split_i}'
        if not experiment_name in os.listdir('lightning_logs') :
            litschnet = BioSchNet()
            logger = TensorBoardLogger(save_dir=os.getcwd(), version=experiment_name, name="lightning_logs")
            early_stopping_callback = EarlyStopping(monitor="val_loss")
            trainer = pl.Trainer(logger=logger, 
                                 max_epochs=20, 
                                 gpus=1,
                                 callbacks=[early_stopping_callback])
            trainer.fit(litschnet, train_loader, val_loader)
            trainer.test(litschnet, test_loader)
            
        # experiment_name = f'{split_type}_split_{split_i}_molsize'
        # if not experiment_name in os.listdir('lightning_logs') :
        #     molsize_model = MolSizeModel()
        #     logger = TensorBoardLogger(save_dir=os.getcwd(), version=experiment_name, name="lightning_logs")
        #     trainer = pl.Trainer(logger=logger, max_epochs=20, gpus=1)
        #     trainer.fit(molsize_model, train_loader, val_loader)
        #     trainer.test(molsize_model, test_loader)

/home/bb596/anaconda3/envs/bioconfpred/lib/python3.7/site-packages/torch/distributed/_sharded_tensor/__init__.py:10: DeprecationWarning: torch.distributed._sharded_tensor will be deprecated, use torch.distributed._shard.sharded_tensor instead
  DeprecationWarning
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type         | Params
--------------------------------------------
0 | schnet     | AtomicSchNet | 455 K 
1 | leaky_relu | LeakyReLU    | 0     
2 | sigmoid    | Sigmoid      | 0     
--------------------------------------------
455 K     Trainable params
0         Non-trainable params
455 K     Total params
1.823     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [ ]:
test

In [ ]:
from collections import defaultdict
n_bioactive_conformations = defaultdict(list)
n_conformations = defaultdict(list)
data_dir = 'data/'

for split in splits :
    
    for iteration in range(5) :
    
        pdbbind_chunks = [filename for filename in os.listdir(os.path.join(data_dir, 'processed')) if filename.startswith('pdbbind')]
        pdbbind_n_chunks = len(pdbbind_chunks)

        train_datasets = []
        val_datasets = []
        test_datasets = []

        if split in ['random', 'scaffold'] :

            with open(os.path.join(data_dir, f'ligand_{split}_splits', f'train_smiles_{split}_split_{iteration}.txt'), 'r') as f :
                train_smiles = f.readlines()
                train_smiles = [smiles.strip() for smiles in train_smiles]

            with open(os.path.join(data_dir, f'ligand_{split}_splits', f'val_smiles_{split}_split_{iteration}.txt'), 'r') as f :
                val_smiles = f.readlines()
                val_smiles = [smiles.strip() for smiles in val_smiles]

            with open(os.path.join(data_dir, f'ligand_{split}_splits', f'test_smiles_{split}_split_{iteration}.txt'), 'r') as f :
                test_smiles = f.readlines()
                test_smiles = [smiles.strip() for smiles in test_smiles]

            for chunk_number in tqdm(range(pdbbind_n_chunks)) :

                dataset = ConfEnsembleDataset(loaded_chunk=chunk_number,
                                              smiles_list=train_smiles)
                train_datasets.append(dataset)

                dataset = ConfEnsembleDataset(loaded_chunk=chunk_number,
                                              smiles_list=val_smiles)
                val_datasets.append(dataset)

                dataset = ConfEnsembleDataset(loaded_chunk=chunk_number,
                                              smiles_list=test_smiles)
                test_datasets.append(dataset)

        else : #protein split

            with open(os.path.join(data_dir, 'protein_similarity_splits', f'train_pdb_protein_similarity_split_{iteration}.txt'), 'r') as f :
                train_pdbs = f.readlines()
                train_pdbs = [pdb.strip() for pdb in train_pdbs]

            with open(os.path.join(data_dir, 'protein_similarity_splits', f'val_pdb_protein_similarity_split_{iteration}.txt'), 'r') as f :
                val_pdbs = f.readlines()
                val_pdbs = [pdb.strip() for pdb in val_pdbs]

            with open(os.path.join(data_dir, 'protein_similarity_splits', f'test_pdb_protein_similarity_split_{iteration}.txt'), 'r') as f :
                test_pdbs = f.readlines()
                test_pdbs = [pdb.strip() for pdb in test_pdbs]

            for chunk_number in tqdm(range(pdbbind_n_chunks)) :

                dataset = ConfEnsembleDataset(loaded_chunk=chunk_number,
                                              pdb_ids_list=train_pdbs)
                train_datasets.append(dataset)

                dataset = ConfEnsembleDataset(loaded_chunk=chunk_number,
                                              pdb_ids_list=val_pdbs)
                val_datasets.append(dataset)

                dataset = ConfEnsembleDataset(loaded_chunk=chunk_number,
                                              pdb_ids_list=test_pdbs)
                test_datasets.append(dataset)

        train_dataset = ConcatDataset(train_datasets)
        val_dataset = ConcatDataset(val_datasets)
        test_dataset = ConcatDataset(test_datasets)
        
        d = {
            'train' : train_dataset,
            'val' : val_dataset,
            'test' : test_dataset
        }
        for s, dataset in d.items() :
            n_conformations[s].append(len(dataset))
            n_bio = 0
            for data in dataset :
                if data.rmsd == 0 :
                    n_bio = n_bio + 1
            n_bioactive_conformations[s].append(n_bio)

In [ ]:
data_dir='data/'
pdbbind_chunks = [filename for filename in os.listdir(os.path.join(data_dir, 'processed')) if filename.startswith('pdbbind')]
pdbbind_n_chunks = len(pdbbind_chunks)
n_confs = 0
n_bio = 0

for chunk_number in range(pdbbind_n_chunks) :
    dataset = ConfEnsembleDataset(loaded_chunk=chunk_number)
    n_confs = n_confs + len(dataset)
    for data in dataset :
        if data.rmsd == 0 :
            n_bio = n_bio + 1
print(n_confs)
print(n_bio)

In [ ]:
n_conformations

In [ ]:
n_bioactive_conformations